# Matrix of metrics of Cross validation  for 5 top Simple Combination 
Date: 09/12/2022
Description: Create cross validation metrics
Use 5 combinations only regarding to the ranking from simple combination result.
This file creates cross validation metrics for next steps
Modificaiton 02/01/2023
Sử dụng 5 combination mới

In [1]:
!nvidia-smi

Sat Feb  4 17:06:20 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 525.60.13    Driver Version: 525.60.13    CUDA Version: 12.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Quadro RTX 6000     On   | 00000000:21:00.0 Off |                  Off |
| 34%   38C    P8    22W / 260W |   1045MiB / 24576MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
# Install packages
!pip install -U scikit-learn
!pip install ftfy
# !pip install xlsxwriter 
# !pip install xlwt


[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip

[notice] A new release of pip available: 22.3.1 -> 23.0
[notice] To update, run: pip install --upgrade pip


In [3]:
# Imports
import tensorflow as tf 
# from google.colab import drive
import os

from tensorflow.keras.models import Sequential, model_from_json
from tensorflow.keras.layers import Dense

import traceback
import sys
import copy
import numpy as np
from itertools import combinations
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestRegressor

# lib for LSTM of Meta model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
import seaborn as sns


import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import TensorDataset, DataLoader
from torch.autograd import Variable

from ftfy import fix_text
import warnings
warnings.simplefilter("ignore", UserWarning)

plt.rcParams['figure.figsize'] = [7, 3]
import math

# Global param and project path
All mix data 1997-2018

In [4]:
# Attach Google Drive for reading and saving files
# drive.mount('/content/drive')
prj_path = '/home/mlworker/Quang/HealthCare/Source_14012023_v4/'

path_all_mix_data =             prj_path+ 'results/all_mix_data/all_mix_data_1997_2018/'
path_all_mix_data_cnn =         path_all_mix_data+ '/cnn/'
path_all_mix_data_lstm =        path_all_mix_data+ '/lstm/'
path_all_mix_data_lstm_att =    path_all_mix_data+ '/lstm_att/'
path_all_mix_data_tf =          path_all_mix_data+ '/transformer/'

prj_path_opt= prj_path + '/optimize_hyperparam/opt_results/optimize_1997_2018/'

prj_path_result_ensemble =      prj_path+ '/results/ensemble/'
com_path_validation_metrics =   prj_path_result_ensemble+ '/result/weighted_combination/cv_metrics/'

com_eval_path_validation_metrics = prj_path + '/eval_result/21_12_2022_weighted_combination-5-cities/'


# loockback_window for meta model
meta_lookback_window = 3

In [5]:
com_path_validation_metrics

'/home/mlworker/Quang/HealthCare/Source_14012023_v4//results/ensemble//result/weighted_combination/cv_metrics/'

# Xây dựng Meta Data Set

In [6]:
##########################
# Xây Dựng Meta Data Set
meta_test_set_city = {}
meta_train_set_city = {}
meta_data_set_city = {}
# cities = ['An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
#         'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau',
#         'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
#         'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
#         'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng',
#         'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
#         'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi',
#         'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
#         'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
#         'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên',
#          'Đà Nẵng', 'Đắk Nông', 'Đắk Lắk', 'Đồng Tháp']
cities = [
     'An Giang', 'BR Vũng Tàu', 'Bình Phước', 'Bình Thuận', 'Bình Định',
#         'Bạc Liêu', 'Bắc Kạn', 'Bắc Giang', 'Cao Bằng', 'Cà Mau','Đà Nẵng',
        
#     'Cần Thơ', 'Gia Lai', 'Hà Giang', 'Hà Nội', 'Hà Tĩnh', 'Hòa Bình',
#         'Hưng Yên', 'Hải Dương', 'Hải Phòng', 'Khánh Hòa', 'Kiên Giang',
        
#     'Kon Tum', 'Lai Châu', 'Long An', 'Lào Cai', 'Lâm Đồng', 'Đắk Nông',
#         'Lạng Sơn', 'Nam Định', 'Nghệ An', 'Ninh Bình', 'Ninh Thuận',
        
#     'Phú Thọ', 'Phú Yên', 'Quảng Bình', 'Quảng Nam', 'Quảng Ngãi', 'Đắk Lắk',
#         'Quảng Ninh', 'Quảng Trị', 'Sóc Trăng', 'Sơn La', 'TT Huế',
        
#     'Thanh Hóa', 'Thái Bình', 'Thái Nguyên', 'Tiền Giang', 'Trà Vinh',
#         'Tuyên Quang', 'Tây Ninh', 'Vĩnh Phúc', 'Yên Bái', 'Điện Biên','Đồng Tháp'
         ]

for city in cities:  
  meta_test_set = pd.DataFrame()
  meta_train_set = pd.DataFrame() 
  #Note: mỗi bộ all_mix_data sẽ có lenght = 251 (10997 -01/01/2018  và vì 1 dòng đầu bị bỏ đi do quá trình statitionary và remove NAN lúc train)
  lenght_whole_data_city = 251
  ######################################
  # Xử lý data của CNN ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_cnn+'mix_train_test_data_'+city+'_cnn.xlsx')
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index() 
  # Tạo meta testset
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
  meta_test_cnn = result_test.reset_index(drop=True)  
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_cnn = result_train.reset_index(drop=True)
  
  ######################################
  # Xử lý data của LSTM  ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_lstm+'mix_train_test_data_'+city+'_lstm.xlsx')  
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
  meta_test_lstm = result_test.reset_index(drop=True)
  # Tạo meta trainset
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_lstm = result_train.reset_index(drop=True)

  ###########################################
  # Xử lý data của LSTM ATT ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_lstm_att+'mix_train_test_data_'+city+'_lstm_att.xlsx') 
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  # Meta testset sẽ có lenght = 36 + lookback_window 
  meta_test_observed = selected_result['Observed'].iloc[-meta_lookback_window-36:]
  meta_test_observed = meta_test_observed.reset_index(drop=True) 

  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
  meta_test_lstm_att = result_test.reset_index(drop=True)  
  
  # Tạo meta trainset
  meta_train_observed = selected_result['Observed'].iloc[meta_lookback_window:-36]
  meta_train_observed = meta_train_observed.reset_index(drop=True)
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_lstm_att = result_train.reset_index(drop=True)


  #############################################
  # Xử lý data của Transformer ứng với mỗi city
  df = pd.read_excel(path_all_mix_data_tf+'mix_train_test_data_'+city+'_tf.xlsx') 
  selected_result = df.sort_values(by = 'rmse', ascending=True) # Sắp xếp theo rmse nhỏ nhất
  selected_result = selected_result.head(lenght_whole_data_city) # lấy 251 phần tử đầu tiên
  selected_result= selected_result.sort_index()   
  
  # Tạo meta testset
  # Meta testset sẽ có lenght = 36 + lookback_window 
  meta_test_observed = selected_result['Observed'].iloc[-meta_lookback_window-36:]
  meta_test_observed = meta_test_observed.reset_index(drop=True) 

  result_test = selected_result['Train Prediction'].iloc[-meta_lookback_window-36:-36] # Lấy buffer cho phần lookback_window
  result_test = result_test.append(selected_result['Test Prediction'].tail(36)) # Lấy 36 tháng cuối
  meta_test_tf = result_test.reset_index(drop=True)  
  
  # Tạo meta trainset
  meta_train_observed = selected_result['Observed'].iloc[meta_lookback_window:-36] # Lấy từ meta_lookback_window cho đến -36 (3 năm cuối)
  meta_train_observed = meta_train_observed.reset_index(drop=True)
  result_train = selected_result['Train Prediction'].iloc[meta_lookback_window:-36]
  meta_train_tf = result_train.reset_index(drop=True)

  
  meta_test_set['cnn_pred'] = meta_test_cnn
  meta_test_set['lstm_pred'] = meta_test_lstm
  meta_test_set['lstm_att_pred'] = meta_test_lstm_att
  meta_test_set['tf_pred'] = meta_test_tf
  meta_test_set['Observed'] = meta_test_observed
   
  meta_train_set['cnn_pred'] = meta_train_cnn
  meta_train_set['lstm_pred'] = meta_train_lstm
  meta_train_set['lstm_att_pred'] = meta_train_lstm_att 
  meta_train_set['tf_pred'] = meta_train_tf  
  meta_train_set['Observed'] = meta_train_observed

  meta_test_set_city[city] = meta_test_set # test set có chiều dài 39
  meta_train_set_city[city] = meta_train_set # train set có chiều dài 212 = 252 -36 (train) - 1 (station) - 3 (meta_lookback_window)

  # Đoạn này làm thêm. Nối dài meta_test_set_city và meta_train_set_city
  # lưu ý đoạn này sẽ tạo ra data có chiều dài = 252 - 1 (bỏ đi 1 tháng đầu do xử lý stationary) - meta_lookback_window
  df = meta_train_set.append(meta_test_set[3:])
  df = df.reset_index(drop=True)
  # range date từ 1 + lookback_window tháng đầu tiên
  
  df['year_month'] = pd.Series(pd.date_range("1997-05-01", periods=248, freq="M"))  
  meta_data_set_city[city] = df
  df.to_excel(prj_path_result_ensemble+'/meta_data/meta_data_'+city+'.xlsx')


<ipython-input-6-6b8437c33104>:47: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-6-6b8437c33104>:62: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-6-6b8437c33104>:81: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36))
<ipython-input-6-6b8437c33104>:104: FutureWarning: The series.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  result_test = result_test.append(selected_result['Test Prediction'].tail(36)) # L

# Danh sách các based Model và các Combination có thể của nó

In [7]:
# Danh sách các based Model và các Combination có thể của nó
list_based_model = ['cnn_pred','lstm_att_pred','lstm_pred','tf_pred']

combs_features = list(combinations(list_based_model[0:], 2)) + list(combinations(list_based_model[0:], 3)) + list(combinations(list_based_model[0:], 4))

# list_based_model = ['cnn_pred','lstm_att_pred','lstm_pred','tf_pred']

# combs_features = list(combinations(list_based_model[0:], 3)) 
combs_features

[('cnn_pred', 'lstm_att_pred'),
 ('cnn_pred', 'lstm_pred'),
 ('cnn_pred', 'tf_pred'),
 ('lstm_att_pred', 'lstm_pred'),
 ('lstm_att_pred', 'tf_pred'),
 ('lstm_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_att_pred', 'lstm_pred'),
 ('cnn_pred', 'lstm_att_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_pred', 'tf_pred'),
 ('lstm_att_pred', 'lstm_pred', 'tf_pred'),
 ('cnn_pred', 'lstm_att_pred', 'lstm_pred', 'tf_pred')]

# Selected 5 Combination

In [8]:
selected_combs = ['LSTM*cnn_pred-lstm_pred-Observed']
def is_in_Selected_combination(comb_string = ''):
    return any(comb_string in x for x in selected_combs)


# Support Functions and Train class 

In [9]:
import requests

def send_to_telegram(message):

    apiToken = '5908735099:AAGVSLrW62aXPBP-GrMvxoVgMsuJxXJpP1Q'
    chatID = '@ptn_announcement'
    apiURL = f'https://api.telegram.org/bot{apiToken}/sendMessage'

    try:
        response = requests.post(apiURL, json={'chat_id': chatID, 'text': message})
        print(response.text)
    except Exception as e:
        print(e)


def seed_everything(seed: int):
    import random, os    
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

seed_everything(42)

# Set hyperparameters as args using the Configuration class
class Configuration():
    def __init__(self):
      # lấy bộ test dài 36 tháng = 3 năm
        self.test_size = 36
        # là nhìn vào dữ liệu trước 3 tháng và dự phóng        
        self.look_back = 3
        # dự phóng 3 tháng
        self.n_predicted_month = 3
        self.n_features = 3
        self.seed = 42
        # mỗi phần tử x trong tập suppervise có độ lớn là 16 = 16 tháng
        self.batch_size = 16
        # self.device = torch.device("cuda")
        self.device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
        self.epochs = 300
     

# Biến args lưu giá trị Global.
args = Configuration()


# Lưu ý sau khi stationnary thì dòng đầu tiên trừ cho dòng trước nữa là ko có cho nên về cơ bản sẽ thành NaN
# Cần remove dòng đầu tiên ra khỏi data để train và test. Cho nên cơ bản là data sau khi in ra sẽ thiếu 1 đơn vị đầu tiên
def convert_to_stationary(city_data):
    """Subtracts previous value for all cols except disease rates"""
    for col_name in city_data.columns:
        if col_name != 'Diarrhoea_rates':
            try:
                city_data[col_name] = city_data[col_name] - city_data[col_name].shift()
            except:
                print(col_name)
    return city_data

def impute_missing_value(city_data):
    """
    Cơ bản dữ liệu bị thiếu sót rất nhiều: Như Điện Biên 1997 -2003 là thiếu dữ liệu về bệnh
    Hàm này sẽ tự sinh ra dữ liệu bị thiếu. Nếu tháng nào không có số liệu thì tính như sau:
    12 tháng đầu không có số liệu thì gán = 0
    tháng 13-24 không có số liệu, sẽ lấy giá trị của tháng cùng kỳ năm trước
    tháng từ 24 trở đi sẽ lấy giá trị nhỏ nhất của 2 tháng cùng kỳ trong 2 năm gần nhất.
    Do Điện Biên bằng 0 nên sau khi xử lý từ 1997 -2003 là đều = 0.  
    """
    for col in city_data.columns:
        for index in range(len(city_data[col])):
            if np.isnan(city_data[col].iloc[index]):
                if index < 12:
                    city_data[col].iloc[index] = 0
                elif index >= 12 and index <= 24:
                    city_data[col].iloc[index] = city_data[col].iloc[index - 12]
                else:
                    city_data[col].iloc[index] = min(city_data[col].iloc[index - 12], city_data[col].iloc[index - 24])
    return city_data

def split_data(data, look_back ):
    """Splits data into train and test sets based on args (Configuration class)"""
    train = data[: -args.test_size]    
    test = data[-args.test_size - look_back: ]
    return train, test

def to_supervised(data, d_in=args.look_back, d_out=args.n_predicted_month, features_list=[]):
    """
    Frames time-series as supervised learning dataset.
    
    Args:
      d_in: lookback window
      d_out: number of predicted months
      features_list: list of all features **where last col is the disease incidence**

    Returns:
      Numpy arrays of disease incidence (y) and other predictors (X)
    """
    X, y = list(), list()
    for index, _ in enumerate(data):
        in_end = index + d_in
        out_end = in_end + d_out
        if out_end <= len(data):
            if len(features_list) == 0 :
                X.append(data[index: in_end, :])
            else:
                X.append(data[index: in_end, features_list])
            y.append(data[in_end: out_end, -1])
    return np.array(X), np.array(y)


def get_data(train_np, test_np, batch_size, list_selected_features, origin_city_meta_data):
    """
    Returns important feature list and data formatted for input into Pytorch 
    models
    """ 
    # origin_city_meta_data chứa data mẫu để lấy index 
    # important_features = arrary index nên ta phải truyền indexex của columns
    important_features = [origin_city_meta_data.columns.get_loc(c) for c in list_selected_features if c in origin_city_meta_data]  

    train_X, train_y = to_supervised(train_np, features_list=important_features)
    test_X, test_y = to_supervised(test_np, features_list=important_features)
    train_tensor = TensorDataset(torch.from_numpy(train_X), torch.from_numpy(train_y))
    test_tensor = (torch.from_numpy(test_X), torch.from_numpy(test_y))

    train_loader = DataLoader(train_tensor, batch_size=batch_size, shuffle=True)

    return important_features, train_loader, test_tensor

#Define Pytorch LSTM model
class MultiVariateLSTM(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = output[:, -1, :]
        y_predicted = self.linear(last_hidden_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            #return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

#Define Pytorch LSTM-ATT model
class MultiVariateLSTM_Attention(nn.Module):
    def __init__(self, n_feature=3, n_layers=2, hidden_size=50):
        super(MultiVariateLSTM_Attention, self).__init__()
        self.lstm = nn.LSTM(input_size=n_feature, hidden_size=hidden_size, num_layers=n_layers, batch_first=True)
        self.attention_linear = nn.Linear(hidden_size, hidden_size)
        # self.linear = nn.Linear(hidden_size*2, args.n_predicted_month)
        self.linear = nn.Linear(hidden_size, args.n_predicted_month)
        self.loss_fn = nn.MSELoss()
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
    
    def forward(self, X_batch, y_batch=None):
        output, (last_hidden, _) = self.lstm(X_batch)
        last_hidden_vector = last_hidden[-1]
        remain_hidden_vector = output
        e_t = remain_hidden_vector.bmm(self.attention_linear(last_hidden_vector).unsqueeze(2)).squeeze(-1)
        alpha_t = F.softmax(e_t, dim=1)
        attenion_vector = remain_hidden_vector.transpose(2, 1).bmm(alpha_t.unsqueeze(2)).squeeze(-1)
        # combine_vector = torch.cat((last_hidden_vector, attenion_vector), dim=1)
        # combine_vector = last_hidden_vector + attenion_vector
        y_predicted = self.linear(attenion_vector)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted
    
    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Define Pytorch CNN model
class MultivariateCNN(nn.Module):
    def __init__(self, num_filters=[100, 100, 100], dropout=0.01):
        super(MultivariateCNN, self).__init__()
        self.loss_fn = loss = nn.MSELoss()
        self.filter_sizes = [1, 2, 3]
        self.conv1d_list = nn.ModuleList([nn.Conv1d(args.n_features, num_filters[i], self.filter_sizes[i]) for i in range(len(self.filter_sizes))])
        self.linear = nn.Linear(np.sum(num_filters), args.n_predicted_month)
        self.dropout = nn.Dropout(dropout)
        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))

    def forward(self, X_batch, y_batch=None):
        X_batch = X_batch.permute(0, 2, 1)  #(batch_size, n_features, n_look_back)
        X_conv_list = [F.relu(conv1d(X_batch)) for conv1d in self.conv1d_list]
        X_pool_list = [F.max_pool1d(x_conv, kernel_size=x_conv.shape[2]) for x_conv in X_conv_list]
        X_fc = torch.cat([x_pool.squeeze(dim=2) for x_pool in X_pool_list], dim=1)
        y_predicted = self.linear(self.dropout(X_fc))
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
            # return y_predicted, self.loss_fn(y_predicted, y_batch)
        else:
            return y_predicted

    def predict(self, X):
        X = torch.tensor(X, device=args.device)
        return self.forward(X)

# Transformer Model
# Transformer Model
class PositionalEncoder(nn.Module):
    def __init__(self, n_feature=3, dropout=0.1, max_length=24):
        super().__init__()
        
        self.hidden_dim = n_feature
        self.max_length = max_length
        self.dropout = nn.Dropout(dropout)
        
        pe = torch.zeros(max_length, self.hidden_dim)
        for pos in range(max_length):
            for i in range(0, self.hidden_dim, 2):
                pe[pos, i] = math.sin(pos/(10000**(2*i/self.hidden_dim)))
                if i + 1 < self.hidden_dim:
                    pe[pos, i+1] = math.cos(pos/(10000**((2*i+1)/self.hidden_dim)))
        pe = pe.unsqueeze(0)        
        self.register_buffer('pe', pe)
    
    def forward(self, x):
        x = x*math.sqrt(self.hidden_dim)
        length = x.size(1)
        pe = Variable(self.pe[:, :length], requires_grad=False)
        if x.is_cuda:
            pe.cuda()
        x = x + pe
        x = self.dropout(x)
        
        return x

class TransformerModel(nn.Module):
    def __init__(self, d_input, n_look_back, hidden_size, n_layers, dropout=0.5):
        super(TransformerModel, self).__init__()
        self.hidden_size = hidden_size

        self.pe = PositionalEncoder(n_feature=args.n_features, dropout=dropout)

        encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=d_input, dim_feedforward=hidden_size, dropout=dropout, activation='gelu')
        # encoder_layers = nn.TransformerEncoderLayer(d_model=d_input, nhead=d_input, dropout=dropout, activation='gelu')
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, n_layers)
        self.decoder = nn.Linear(d_input*n_look_back, args.n_predicted_month)

        self.sigma = nn.Parameter(torch.ones(args.n_predicted_month))
        
        self.loss_fn = nn.MSELoss()
        
    
    def forward(self, X_batch, y_batch=None):
        X_batch = self.pe(X_batch)
        X_batch = self.transformer_encoder(X_batch)
        X_batch = X_batch.view(X_batch.size(0), -1)
        
        y_predicted = self.decoder(X_batch)
        if y_batch != None:
            assert y_predicted.size() == y_batch.size()
            loss = self.loss_fn(y_predicted, y_batch)
            loss = 0.5 * loss / self.sigma**2
            loss = loss.sum() + torch.log(1 + self.sigma.prod())
            return y_predicted, loss
        else:
            return y_predicted
        return X_batch
    
    def predict(self, X):
        with torch.no_grad():
            X = torch.tensor(X, device=args.device)
        return self.forward(X).squeeze()

# Create class to train and evaluate models
class Trainer():
    def __init__(self, city, model_type, learning_rate, important_features, train_loader, test_tensor, n_layers=2, hidden_size=128, num_filters=[100, 100, 100], dropout=0.01):
        """
        Initialise trainer, allowing input of LSTM, LSTM-ATT, or CNN 
        hyperparameters. Adam optimiser used for all models.
        """
        self.model_type = model_type    
        self.city = city    
        self.model = self.init_model(model_type, n_layers, hidden_size, num_filters, dropout, city)
        self.model.double().to(args.device)
        self.optimizer = torch.optim.Adam(self.model.parameters(), lr=learning_rate)
        self.important_features, self.train_loader, self.test_tensor = important_features, train_loader, test_tensor
    
    def init_model(self, model_type, n_layers, hidden_size, num_filters, dropout, city):        
        #Get hyper param by name and set for n_layers, hidden_size, num_filters, dropout and overwrite the input param  
        # n_layers, hidden_size,num_filters, dropout= get_opt_param_by_name( algo = model_type, city = city)
    
        """Initialise a model based on whether LSTM, LSTM-ATT, or CNN is chosen."""
        if model_type.lower() == 'lstm':
            model = MultiVariateLSTM(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'lstm_attention':
            model = MultiVariateLSTM_Attention(args.n_features, n_layers, hidden_size)
        elif model_type.lower() == 'cnn':
            model = MultivariateCNN(num_filters, dropout)
        elif model_type.lower() == 'transformers':
            model = TransformerModel(d_input=args.n_features, n_look_back=args.look_back, hidden_size=hidden_size, n_layers=n_layers, dropout=dropout)
        return model

    def step(self, batch):
        self.model.train()
        X_batch, y_batch = tuple(t.to(args.device) for t in batch)
        self.optimizer.zero_grad()
        y_pred, loss = self.model.forward(X_batch, y_batch)
        loss.backward()
        self.optimizer.step()
        return loss.mean().item()

    def validation(self):
        self.model.eval()
        eval_loss = 0.0

        result = {}

        y_true = np.array([])
        y_pred = np.array([])

        X_batch, y_batch = tuple(t.to(args.device) for t in self.test_tensor)
        with torch.no_grad():
            outputs, loss = self.model.forward(X_batch, y_batch)
            eval_loss = loss.mean().item()

        return eval_loss

    def train(self, epochs=20):
        # print('param of model')
        # print('hidensize',self.model.hidden_size)
        # print('nlayer',self.model.n_layers)  
        best_lost = float("inf")
        best_model = None
        for epoch in range(epochs):
            total_loss = 0.0
            for batch in self.train_loader:
                loss = self.step(batch)
                total_loss += loss
            train_loss = total_loss/len(self.train_loader)
            eval_loss = self.validation()
            if eval_loss < best_lost:
                best_lost = eval_loss
                best_model = copy.deepcopy(self.model)
            if (epoch + 1) == epochs or (epoch + 1) in [c + 1 for c in range(epochs) if c % int(epochs/4) == 0]:
                print(f"Epoch: {epoch:2}/{epochs:2} - train_loss: {train_loss:.4f} - test_loss: {eval_loss:4f}")
        self.model = best_model
        self.model.eval()
        return None



    # Lưu model vào trong thư mục models
    def save_model_to(self, path = '', city =''):       
        torch.save(self.model, path)

    def load_model_to(self, path = ''):       
        return torch.load(path)

    def evaluate_model(self, np_data=None, plot=True, scaled=True, city=None, k_steps=None, y_scaler =None):
        assert scaled, "data must be scaled"
        self.model.eval()
        tensor_data = torch.from_numpy(np_data)
        rmse_list = []
        mae_list = [] 
        mape_list = []

        y_predicted_list = []
        y_true_list = []

        for k_steps in range(1, args.n_predicted_month + 1):
            y_predicted = []
            y_true = []
            for index in range(tensor_data.size(0) - args.look_back):
                X = tensor_data[index: index + args.look_back, self.important_features]
                # yhat = self.model.predict(X.unsqueeze(0)).squeeze()

                yhat = self.model.predict(X.unsqueeze(0))
                yhat = yhat.squeeze()

                y_predicted.append(yhat.detach().cpu().numpy()[k_steps - 1])
                y_true.append(tensor_data[index + args.look_back, -1].detach().cpu().numpy())

            y_predicted = y_scaler.inverse_transform(np.array(y_predicted).reshape(-1, 1)).reshape(-1, )
            y_true = y_scaler.inverse_transform(np.array(y_true).reshape(-1, 1)).reshape(-1, )

            

            rmse = mean_squared_error(y_true, y_predicted, squared=False)
            mae = mean_absolute_error(y_true, y_predicted)
            mape = mean_absolute_percentage_error(y_true, y_predicted)

            rmse_list.append(rmse)
            mae_list.append(mae)
            mape_list.append(mape)

            print('City: '+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))          
            if plot==True:
              plt.grid(True)
              plt.plot(y_predicted, label='predicted')
              plt.plot(y_true, label='actual')
              plt.title(f"k-steps = {k_steps} - city: "+self.city+'  _algo:'+self.model_type+'  -RMSE: '+str(rmse))
              plt.legend()
              plt.show()

              plt.show()
            y_predicted_list.append(y_predicted)
            y_true_list.append(y_true)

        return y_true_list, y_predicted_list, rmse_list, mae_list, mape_list

# Get optimized Hyper param

In [10]:
def convert_str_num_filter_array(str_filter = ''):  
  arr_filter = str_filter.split(',')  
  int_arr_filter= [int(x) for x in arr_filter]
  return int_arr_filter


opt_param_cnn = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_cnn.xlsx', index_col = 'City')
opt_param_lstm = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_lstm.xlsx', index_col = 'City')
opt_param_lstm_att = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_lstm_att.xlsx', index_col = 'City')
opt_param_transformer = pd.read_excel(prj_path_opt+'diarrhoea_opt_hyperparam_transformer.xlsx', index_col = 'City')

def get_opt_param_by_name(algo = 'transformers', city = None):
  if algo == 'cnn':
    city_param = opt_param_cnn.loc[city]
    #load  param
    n_layers = ''
    hidden_size = ''
    num_filters = convert_str_num_filter_array(city_param['Num. filters'])
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm':
    city_param = opt_param_lstm.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'lstm_attention':
    city_param = opt_param_lstm_att.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hiden Size'])
    num_filters = ''
    dropout = ''
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  elif algo == 'transformers':
    city_param = opt_param_transformer.loc[city]
    #load  param
    n_layers = int(city_param['n Layers'])
    hidden_size = int(city_param['Hidden Size'])
    num_filters = ''
    dropout = float(city_param['Dropout rate'])
    learning_rate = float(city_param['Learning rate'])
    epoch = int(city_param['Epochs'])
    lookback_window = int(city_param['Lookback Window'])
  return n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window


# Manual Check param before train
n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window= get_opt_param_by_name(algo='transformers', city='Cao Bằng')
lookback_window,epoch, hidden_size, n_layers,learning_rate, num_filters, dropout

(3, 270, 36, 6, 0.001205121044593835, '', 0.4761287407623291)

# Create Cross validation Train/Test set

In [11]:
# tạo l_train_indices\l_test_indices
data = list(range(212))
min_train_size = 72 # Chiều dài ban đầu train size = 6 năm = 72
test_size = 36 # chiều dài test size 36 tháng
l_train_indices= []
l_test_indices = []
for i in range(min_train_size, len(data)-test_size+1):
    train = np.array(data[0:i])
    test = np.array(data[i:i+test_size])    
    l_train_indices.append(train)
    l_test_indices.append(test)

len(l_test_indices)

105

# Function Tạo và Predict. Important (epoch = 1 for test)

In [12]:
def do_train_predict(city ='', algo = '', list_selected_features = '', origin_city_meta_test_set= '', y_scaler = '', s_selected_features= '', train ='', test ='', sample_idx ='', combination_idx= ''):
    
    metric_reslt = pd.DataFrame()
    n_layers, hidden_size, num_filters, dropout, learning_rate, epoch, lookback_window = get_opt_param_by_name( algo = algo, city = city)
    # Get data to run model
    important_features, train_loader, test_tensor = get_data(train, test, args.batch_size, list_selected_features, origin_city_meta_test_set)   

#     epoch = 1 # comment when do experiment
    
    if( algo =='cnn'): 
        algo = 'CNN'    # upper case only
        trainer = Trainer(
                            model_type='cnn',  
                            city = city,                    
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            num_filters= num_filters, 
                            dropout=dropout )

    if( algo =='lstm'):
        algo = 'LSTM'    # upper case only        
        trainer = Trainer(
                            model_type='lstm', 
                            city = city,
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers=n_layers,
                            hidden_size=hidden_size)

    if( algo =='lstm_attention'): 
        algo = 'LSTM ATT'    # upper case only       
        trainer = Trainer(
                            model_type= 'lstm_attention',
                            city = city,
                            learning_rate= learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers= n_layers,
                            hidden_size=hidden_size )

    if( algo =='transformers'): 
        algo = 'TF'    # upper case only       
        trainer = Trainer(
                            model_type='transformers',
                            city = city, 
                            learning_rate=learning_rate,
                            important_features=important_features,
                            train_loader=train_loader,
                            test_tensor=test_tensor,
                            n_layers=n_layers,
                            hidden_size=hidden_size,
                            dropout=dropout)

    
    # Train model with province-specific epochs
    trainer.train(epochs=epoch)  
    # Evaluate test set   
    y_true_test, y_pred_test, rmse_list, mae_list, mape_list = trainer.evaluate_model(np_data=test, plot= False, scaled=True, city=city, y_scaler= y_scaler)
        
    metric_reslt = pd.DataFrame({
                                'City': city,
                                'Meta_Model_Name': algo, 
                                'sample_index':sample_idx,
                                'Based_Models': s_selected_features,                                                      
                                # algo+'_'+str(combination_idx)+'_rmse': rmse_list[0],
                                # algo+'_'+str(combination_idx)+'_mae': mae_list[0]
                                algo+'*'+s_selected_features+'_rmse': rmse_list[0],
                                algo+'*'+s_selected_features+'_mae': mae_list[0]
                                }, index=[0]) # do có 1 dòng nên pandas cần index = 0    
  

        
    return metric_reslt

# Main Cross Validation Metrics Creation
Tạo ra 105 bộ validation. Xuất ra Metrics (RMSE và MAE) cho 5 bộ giải thuật chọn lựa. (10 cột cho 1 dòng validation )

In [13]:
dt_started = datetime.now()

l_errCity = {} # có lỗi sẽ lưu vào đây, kiểm tra ngay cell sau

for city in cities:
  pd_all_rows_new_meta_data = pd.DataFrame()

  for sample_index in  range(0,len(l_train_indices)):

    pd_one_row_new_meta_data = pd.DataFrame()
    combination_idx = 0
    for combination_features in combs_features:            
      list_selected_features = [x for x in combination_features] +['Observed'] # Lấy danh sách tên các features + cột Observed 
      s_selected_features = '-'.join([str(elem) for elem in list_selected_features])      

      # Có trong danh sách chọn lựa thì thực hiện dự đoán
      if((is_in_Selected_combination('CNN*'+s_selected_features)==True) or (is_in_Selected_combination('LSTM*'+s_selected_features) == True)):        
        args.n_features = len(list_selected_features) 
        args.look_back = meta_lookback_window  
        cnn_metrics = pd.DataFrame()
        lstm_metrics = pd.DataFrame()

        try:
          # Lấy data từ train set và chia ra theo valiation
          origin_city_meta_train_set = meta_train_set_city[city]
          origin_city_meta_test_set = meta_test_set_city[city]
          
          train = origin_city_meta_train_set.iloc[l_train_indices[sample_index]]
          test = origin_city_meta_train_set.iloc[l_test_indices[sample_index]]        

          # Data train/test processing for all Meta model
          train = train.astype(float)
          test = test.astype(float)

          train = train.replace((np.inf, -np.inf, np.nan), 0)
          test = test.replace((np.inf, -np.inf, np.nan), 0) 
        
          # Đoạn này impute các trường hợp rỗng
          train = impute_missing_value(train)
          test = impute_missing_value(test)

          # Fit data scaler to training data
          full_scaler = MinMaxScaler().fit(train)
          y_scaler = MinMaxScaler().fit(train.values[:, -1].reshape(-1, 1))

          # Scale train and test data
          train = full_scaler.transform(train)
          test = full_scaler.transform(test)

          if(is_in_Selected_combination('CNN*'+s_selected_features)==True):
            ########################################
            ########## CNN 
            ########################################
            cnn_metrics = do_train_predict(city =city, algo='cnn', list_selected_features = list_selected_features,
                                                origin_city_meta_test_set= origin_city_meta_test_set, y_scaler = y_scaler, 
                                                s_selected_features= s_selected_features, train = train, test = test, 
                                                sample_idx = sample_index,combination_idx= combination_idx)

          if(is_in_Selected_combination('LSTM*'+s_selected_features)==True):
            ########################################
            ########## LSTM 
            ########################################
            lstm_metrics = do_train_predict(city =city, algo='lstm', list_selected_features = list_selected_features,
                                                origin_city_meta_test_set= origin_city_meta_test_set, y_scaler = y_scaler, 
                                                s_selected_features= s_selected_features, train = train, test = test, 
                                                sample_idx = sample_index,combination_idx= combination_idx)
          

          
          
          ###############################   
          ## Khúc này rất thủ công
          if(cnn_metrics.empty==False and lstm_metrics.empty ==False):              
            join_2_results = pd.concat([cnn_metrics.iloc[:, [4, 5]], lstm_metrics.iloc[:, [4, 5]]], axis=1)
          
          if(cnn_metrics.empty== False and lstm_metrics.empty ==True):
            join_2_results = cnn_metrics.iloc[:, [4, 5]]
          
          if(cnn_metrics.empty== True and lstm_metrics.empty ==False):
            join_2_results = lstm_metrics.iloc[:, [4, 5]]

          if(pd_one_row_new_meta_data.empty):
            pd_one_row_new_meta_data = join_2_results
          else:
            pd_one_row_new_meta_data = pd.concat([join_2_results, pd_one_row_new_meta_data], axis=1)       

          combination_idx = combination_idx + 1
        except Exception as e:
          l_errCity[city] = e
          break    

        
    # Hết vòng lập pd_one_row_new_meta_data chứa 5 bộ kết quả của 1 sameple    
    # Qua sample kế tiếp   
    pd_all_rows_new_meta_data = pd.concat([pd_all_rows_new_meta_data, pd_one_row_new_meta_data], axis=0) 

  
  # Kết thúc hết tất cả validation -> Lưu kết quả cuối cùng ứng với từng thành phố
  pd_all_rows_new_meta_data.to_excel(com_path_validation_metrics+ city+'_5_comb_cv_metrics.xlsx')
  
  # send_to_telegram('Server Chạy city: '+ city)

dt_ended = datetime.now()
run_time = round((dt_ended - dt_started).total_seconds()/60)
print('kết thúc train trong:', run_time)
print ('danh sách error city: ')
print(l_errCity)
# Telegram vào

Epoch:  0/380 - train_loss: 0.8158 - test_loss: 0.749400
Epoch: 95/380 - train_loss: 0.2249 - test_loss: 0.130460
Epoch: 190/380 - train_loss: 0.1920 - test_loss: 0.125897
Epoch: 285/380 - train_loss: 0.1466 - test_loss: 0.156104
Epoch: 379/380 - train_loss: 0.1141 - test_loss: 0.224683
City: An Giang  _algo:lstm  -RMSE: 34.70422862296005
City: An Giang  _algo:lstm  -RMSE: 25.07752372181139
City: An Giang  _algo:lstm  -RMSE: 31.47032705366733
Epoch:  0/380 - train_loss: 0.8309 - test_loss: 0.714721
Epoch: 95/380 - train_loss: 0.2265 - test_loss: 0.135557
Epoch: 190/380 - train_loss: 0.1710 - test_loss: 0.157470
Epoch: 285/380 - train_loss: 0.0950 - test_loss: 0.389484
Epoch: 379/380 - train_loss: 0.0500 - test_loss: 0.499291
City: An Giang  _algo:lstm  -RMSE: 30.463403417925132
City: An Giang  _algo:lstm  -RMSE: 35.42036201425969
City: An Giang  _algo:lstm  -RMSE: 53.14184579981366
Epoch:  0/380 - train_loss: 0.8214 - test_loss: 0.738516
Epoch: 95/380 - train_loss: 0.2249 - test_loss: 

Epoch: 285/380 - train_loss: 0.0796 - test_loss: 0.395611
Epoch: 379/380 - train_loss: 0.1151 - test_loss: 0.140595
City: An Giang  _algo:lstm  -RMSE: 22.22707343223101
City: An Giang  _algo:lstm  -RMSE: 25.194245405778464
City: An Giang  _algo:lstm  -RMSE: 17.150670576866112
Epoch:  0/380 - train_loss: 0.7764 - test_loss: 0.711074
Epoch: 95/380 - train_loss: 0.1919 - test_loss: 0.108124
Epoch: 190/380 - train_loss: 0.1543 - test_loss: 0.091880
Epoch: 285/380 - train_loss: 0.1044 - test_loss: 0.165780
Epoch: 379/380 - train_loss: 0.0396 - test_loss: 0.431688
City: An Giang  _algo:lstm  -RMSE: 22.283772814930924
City: An Giang  _algo:lstm  -RMSE: 32.30549741812392
City: An Giang  _algo:lstm  -RMSE: 19.91027523997718
Epoch:  0/380 - train_loss: 0.7500 - test_loss: 0.768072
Epoch: 95/380 - train_loss: 0.2178 - test_loss: 0.124837
Epoch: 190/380 - train_loss: 0.1462 - test_loss: 0.210583
Epoch: 285/380 - train_loss: 0.0528 - test_loss: 0.464638
Epoch: 379/380 - train_loss: 0.0286 - test_lo

City: An Giang  _algo:lstm  -RMSE: 24.892139594777344
Epoch:  0/380 - train_loss: 0.7338 - test_loss: 0.646948
Epoch: 95/380 - train_loss: 0.1897 - test_loss: 0.099670
Epoch: 190/380 - train_loss: 0.1697 - test_loss: 0.089454
Epoch: 285/380 - train_loss: 0.0615 - test_loss: 0.074603
Epoch: 379/380 - train_loss: 0.0207 - test_loss: 0.199538
City: An Giang  _algo:lstm  -RMSE: 19.418114834419363
City: An Giang  _algo:lstm  -RMSE: 15.023713673039794
City: An Giang  _algo:lstm  -RMSE: 18.608584621914005
Epoch:  0/380 - train_loss: 0.7360 - test_loss: 0.657169
Epoch: 95/380 - train_loss: 0.1631 - test_loss: 0.102834
Epoch: 190/380 - train_loss: 0.1281 - test_loss: 0.069072
Epoch: 285/380 - train_loss: 0.1262 - test_loss: 0.100398
Epoch: 379/380 - train_loss: 0.0214 - test_loss: 0.415720
City: An Giang  _algo:lstm  -RMSE: 15.704779047212774
City: An Giang  _algo:lstm  -RMSE: 15.921475284458973
City: An Giang  _algo:lstm  -RMSE: 17.17449672515175
Epoch:  0/380 - train_loss: 0.7825 - test_loss:

Epoch: 95/380 - train_loss: 0.1538 - test_loss: 0.070929
Epoch: 190/380 - train_loss: 0.1184 - test_loss: 0.063694
Epoch: 285/380 - train_loss: 0.0493 - test_loss: 0.033369
Epoch: 379/380 - train_loss: 0.0139 - test_loss: 0.029780
City: An Giang  _algo:lstm  -RMSE: 11.489852324678163
City: An Giang  _algo:lstm  -RMSE: 13.336755463306304
City: An Giang  _algo:lstm  -RMSE: 14.795031266628722
Epoch:  0/380 - train_loss: 0.7562 - test_loss: 0.645932
Epoch: 95/380 - train_loss: 0.1759 - test_loss: 0.081350
Epoch: 190/380 - train_loss: 0.1139 - test_loss: 0.061811
Epoch: 285/380 - train_loss: 0.0427 - test_loss: 0.039833
Epoch: 379/380 - train_loss: 0.0798 - test_loss: 0.042262
City: An Giang  _algo:lstm  -RMSE: 16.244654025738992
City: An Giang  _algo:lstm  -RMSE: 12.27227284556723
City: An Giang  _algo:lstm  -RMSE: 13.420552699676271
Epoch:  0/380 - train_loss: 0.7251 - test_loss: 0.638336
Epoch: 95/380 - train_loss: 0.1711 - test_loss: 0.085387
Epoch: 190/380 - train_loss: 0.1102 - test_l

Epoch: 285/380 - train_loss: 0.0393 - test_loss: 0.654841
Epoch: 379/380 - train_loss: 0.0248 - test_loss: 0.724700
City: An Giang  _algo:lstm  -RMSE: 14.937292729755448
City: An Giang  _algo:lstm  -RMSE: 24.90224237293672
City: An Giang  _algo:lstm  -RMSE: 33.34693999850094
Epoch:  0/380 - train_loss: 0.7299 - test_loss: 0.637603
Epoch: 95/380 - train_loss: 0.1579 - test_loss: 0.126466
Epoch: 190/380 - train_loss: 0.1063 - test_loss: 0.121604
Epoch: 285/380 - train_loss: 0.0360 - test_loss: 0.624903
Epoch: 379/380 - train_loss: 0.0153 - test_loss: 1.022650
City: An Giang  _algo:lstm  -RMSE: 21.689087963220743
City: An Giang  _algo:lstm  -RMSE: 19.548307555021736
City: An Giang  _algo:lstm  -RMSE: 26.4441463709594
Epoch:  0/380 - train_loss: 0.7353 - test_loss: 0.614183
Epoch: 95/380 - train_loss: 0.1298 - test_loss: 0.113717
Epoch: 190/380 - train_loss: 0.0891 - test_loss: 0.265492
Epoch: 285/380 - train_loss: 0.1017 - test_loss: 0.245139
Epoch: 379/380 - train_loss: 0.0158 - test_los

...

Epoch: 240/480 - train_loss: 0.1678 - test_loss: 0.076050
Epoch: 360/480 - train_loss: 0.1131 - test_loss: 0.072496
Epoch: 479/480 - train_loss: 0.0509 - test_loss: 0.056481
City: Bình Định  _algo:lstm  -RMSE: 6.417648489034185
City: Bình Định  _algo:lstm  -RMSE: 5.7918642141232075
City: Bình Định  _algo:lstm  -RMSE: 6.508930429818906
Epoch:  0/480 - train_loss: 0.7611 - test_loss: 0.690919
Epoch: 120/480 - train_loss: 0.1720 - test_loss: 0.117311
Epoch: 240/480 - train_loss: 0.1496 - test_loss: 0.087809
Epoch: 360/480 - train_loss: 0.1258 - test_loss: 0.092933
Epoch: 479/480 - train_loss: 0.0716 - test_loss: 0.083795
City: Bình Định  _algo:lstm  -RMSE: 8.737962335524237
City: Bình Định  _algo:lstm  -RMSE: 10.779234800829418
City: Bình Định  _algo:lstm  -RMSE: 8.675672625725394
Epoch:  0/480 - train_loss: 0.7226 - test_loss: 0.679370
Epoch: 120/480 - train_loss: 0.1377 - test_loss: 0.069696
Epoch: 240/480 - train_loss: 0.0642 - test_loss: 0.048461
Epoch: 360/480 - train_loss: 0.0566 - 

Epoch: 479/480 - train_loss: 0.0508 - test_loss: 0.050574
City: Bình Định  _algo:lstm  -RMSE: 6.327850908340735
City: Bình Định  _algo:lstm  -RMSE: 6.729537127803957
City: Bình Định  _algo:lstm  -RMSE: 6.337043347787125
Epoch:  0/480 - train_loss: 0.7540 - test_loss: 0.693983
Epoch: 120/480 - train_loss: 0.1483 - test_loss: 0.067585
Epoch: 240/480 - train_loss: 0.0827 - test_loss: 0.054506
Epoch: 360/480 - train_loss: 0.0657 - test_loss: 0.043815
Epoch: 479/480 - train_loss: 0.0519 - test_loss: 0.040194
City: Bình Định  _algo:lstm  -RMSE: 5.468296727636016
City: Bình Định  _algo:lstm  -RMSE: 5.628989293746781
City: Bình Định  _algo:lstm  -RMSE: 5.813645425702251
Epoch:  0/480 - train_loss: 0.7842 - test_loss: 0.710777
Epoch: 120/480 - train_loss: 0.1129 - test_loss: 0.062556
Epoch: 240/480 - train_loss: 0.0640 - test_loss: 0.045985
Epoch: 360/480 - train_loss: 0.0595 - test_loss: 0.043012
Epoch: 479/480 - train_loss: 0.0599 - test_loss: 0.050425
City: Bình Định  _algo:lstm  -RMSE: 5.31

Epoch:  0/480 - train_loss: 0.7076 - test_loss: 0.667426
Epoch: 120/480 - train_loss: 0.1233 - test_loss: 0.061157
Epoch: 240/480 - train_loss: 0.1081 - test_loss: 0.058308
Epoch: 360/480 - train_loss: 0.0642 - test_loss: 0.048325
Epoch: 479/480 - train_loss: 0.0542 - test_loss: 0.052068
City: Bình Định  _algo:lstm  -RMSE: 6.383786978793705
City: Bình Định  _algo:lstm  -RMSE: 6.603947325055636
City: Bình Định  _algo:lstm  -RMSE: 6.708185463520538
Epoch:  0/480 - train_loss: 0.7276 - test_loss: 0.671092
Epoch: 120/480 - train_loss: 0.0980 - test_loss: 0.060918
Epoch: 240/480 - train_loss: 0.0899 - test_loss: 0.057128
Epoch: 360/480 - train_loss: 0.0844 - test_loss: 0.054260
Epoch: 479/480 - train_loss: 0.0761 - test_loss: 0.057290
City: Bình Định  _algo:lstm  -RMSE: 6.065254482860855
City: Bình Định  _algo:lstm  -RMSE: 7.388103645367711
City: Bình Định  _algo:lstm  -RMSE: 8.517375553325413
Epoch:  0/480 - train_loss: 0.7799 - test_loss: 0.700889
Epoch: 120/480 - train_loss: 0.1277 - tes

Epoch: 240/480 - train_loss: 0.0801 - test_loss: 0.049697
Epoch: 360/480 - train_loss: 0.1371 - test_loss: 0.060933
Epoch: 479/480 - train_loss: 0.0502 - test_loss: 0.046716
City: Bình Định  _algo:lstm  -RMSE: 5.758770083149897
City: Bình Định  _algo:lstm  -RMSE: 5.770049042433547
City: Bình Định  _algo:lstm  -RMSE: 5.89251344560556
Epoch:  0/480 - train_loss: 0.7359 - test_loss: 0.669169
Epoch: 120/480 - train_loss: 0.1006 - test_loss: 0.059315
Epoch: 240/480 - train_loss: 0.0977 - test_loss: 0.058312
Epoch: 360/480 - train_loss: 0.0752 - test_loss: 0.055801
Epoch: 479/480 - train_loss: 0.0649 - test_loss: 0.052772
City: Bình Định  _algo:lstm  -RMSE: 5.579453868061445
City: Bình Định  _algo:lstm  -RMSE: 5.572559981787773
City: Bình Định  _algo:lstm  -RMSE: 5.740782096542154
Epoch:  0/480 - train_loss: 0.7497 - test_loss: 0.675135
Epoch: 120/480 - train_loss: 0.1114 - test_loss: 0.059004
Epoch: 240/480 - train_loss: 0.1287 - test_loss: 0.056854
Epoch: 360/480 - train_loss: 0.0815 - tes

Epoch: 479/480 - train_loss: 0.0563 - test_loss: 0.049055
City: Bình Định  _algo:lstm  -RMSE: 4.635938863003863
City: Bình Định  _algo:lstm  -RMSE: 4.307522201894271
City: Bình Định  _algo:lstm  -RMSE: 4.335713965041247
Epoch:  0/480 - train_loss: 0.7185 - test_loss: 0.665374
Epoch: 120/480 - train_loss: 0.0997 - test_loss: 0.048526
Epoch: 240/480 - train_loss: 0.0634 - test_loss: 0.039246
Epoch: 360/480 - train_loss: 0.0566 - test_loss: 0.046596
Epoch: 479/480 - train_loss: 0.1122 - test_loss: 0.054161
City: Bình Định  _algo:lstm  -RMSE: 3.9520892182019773
City: Bình Định  _algo:lstm  -RMSE: 4.165017529402385
City: Bình Định  _algo:lstm  -RMSE: 3.7905704557203794
Epoch:  0/480 - train_loss: 0.7150 - test_loss: 0.683386
Epoch: 120/480 - train_loss: 0.1040 - test_loss: 0.055347
Epoch: 240/480 - train_loss: 0.1134 - test_loss: 0.057171
Epoch: 360/480 - train_loss: 0.0619 - test_loss: 0.036095
Epoch: 479/480 - train_loss: 0.0558 - test_loss: 0.029959
City: Bình Định  _algo:lstm  -RMSE: 4.

In [14]:
send_to_telegram("Server Chạy Xong ensemble weighted " )

{"ok":true,"result":{"message_id":264,"sender_chat":{"id":-1001712314864,"title":"PTN_Announcement","username":"ptn_announcement","type":"channel"},"chat":{"id":-1001712314864,"title":"PTN_Announcement","username":"ptn_announcement","type":"channel"},"date":1675554552,"text":"Server Ch\u1ea1y Xong ensemble weighted"}}
